In [5]:
import pandas as pd
import bs4
import selenium
import requests
import re
from selenium.common.exceptions import NoSuchElementException

# Building Recommendation System

In [6]:
from selenium import webdriver

In [7]:
browser = webdriver.Chrome('chromedriver.exe')
browser.get('https://www.zomato.com/bangalore/great-food-no-bull')

In [8]:
restaurants =browser.find_elements_by_css_selector('div.col-s-8.col-l-1by3')
print(len(restaurants))

132


In [9]:
def get_info(review, selector , attribute):
    try:    
        tag = review.find_element_by_css_selector(selector)
    
        tag = tag.get_attribute(attribute)
    except:
        tag=None
    return tag
df_restaurants=pd.DataFrame(columns={'rest_name','rest_link'})
for restaurant in restaurants:  
    rest_name = get_info(restaurant,'div.res_title.zblack.bold.nowrap','innerHTML')
    if rest_name != None:
        rest_name = rest_name.strip()
    rest_link = get_info(restaurant,'a.relative.top-res-box-bg.pl10.ptop0','href')
    curr_restaurant={
                 'rest_name':rest_name,
                 'rest_link':rest_link
             }
    df_restaurants= df_restaurants.append(curr_restaurant,ignore_index=True)
df_restaurants.head()

,rest_name,rest_link
0,Brewklyn Microbrewery,https://www.zomato.com/bangalore/brewklyn-micr...
1,Output Bengaluru,https://www.zomato.com/bangalore/output-bengal...
2,Punjab Bistro,https://www.zomato.com/bangalore/punjab-bistro...
3,The Old Fashioned Bar,https://www.zomato.com/bangalore/the-old-fashi...
4,1131 Bar + Kitchen,https://www.zomato.com/bangalore/1131-bar-kitc...


In [55]:
df_reviews=pd.DataFrame(columns={'rating','user_name','user_id','rest_name'})

In [56]:
for link in df_restaurants['rest_link']:
    browser.get(link )
    #review_sel ='a.item.respageMenu-item.restaurant-tab-item-jumbo-track.active'
    #review_button = browser.find_element_by_css_selector(review_sel)
    #review_button.click()
    while True:
        more_sel = 'div.load-more'
        try:
            load_more_button = browser.find_element_by_css_selector(more_sel)
            load_more_button.click()

            reviews = browser.find_elements_by_css_selector('div.ui.segment.clearfix.brtop')
        except Exception:
            break
    def get_info(review, selector , attribute):
        try:    
            tag = review.find_element_by_css_selector(selector)

            tag = tag.get_attribute(attribute)
        except:
            tag=None
        return tag
        
    for review in reviews:  
            rating=get_info(review,'div.ttupper','aria-label')
            #y=re.findall('[0-9.]+',rating)
            #rating = float(y[0])
            name_selec = 'a[data-entity_id]'
            user_id =get_info(review,name_selec,'data-entity_id')  
            user_name = get_info(review,name_selec,'innerHTML') 
            if user_name !=None:
                user_name = user_name.strip()
            rest_name = get_info(browser,'a.ui.large.header','title') 
            curr_review={
                         'user_name':user_name,
                         'rating':rating,
                         'user_id':user_id,
                         'rest_name':rest_name
                     }
            df_reviews= df_reviews.append(curr_review,ignore_index=True)
df_reviews.head()  

,rest_name,rating,user_id,user_name
0,Brewklyn Microbrewery Bangalore,Rated 5.0,1826583,Reedhika Puliani
1,Brewklyn Microbrewery Bangalore,Rated 5.0,31021666,Nikhil Singla
2,Brewklyn Microbrewery Bangalore,Rated 5.0,3048280,Alexandrina Namrata
3,Brewklyn Microbrewery Bangalore,Rated 4.0,36088032,Abhishek Sharma
4,Brewklyn Microbrewery Bangalore,Rated 4.0,407847,Nitin Hajela (TheFoodieTales.com)


In [78]:
df_reviews.shape[0]

7974

In [63]:
df_reviews.to_csv('zomato.csv',index=False)

In [1]:
import pandas as pd
zomato = pd.read_csv('zomato.csv')

# Removing the null values

In [2]:
zomato=zomato.dropna(axis=0)

In [3]:
zomato.isnull().sum()

rest_name    0
rating       0
user_id      0
user_name    0
dtype: int64

# Converting the rating column into a numerical value

In [4]:
import re
list1=[]
for i in zomato.rating.items():
    value = re.findall('[0-9.]+',i[1])
    list1.append(value[0])

In [5]:
zomato.rating=list1

In [6]:
zomato.rating = zomato.rating.astype(float)

In [7]:
zomato.user_id = zomato.user_id.astype(int)

In [8]:
zomato.user_id = 'User-' + zomato.user_id.astype(str)
zomato.head()

,rest_name,rating,user_id,user_name
0,Brewklyn Microbrewery Bangalore,5.0,User-1826583,Reedhika Puliani
1,Brewklyn Microbrewery Bangalore,5.0,User-31021666,Nikhil Singla
2,Brewklyn Microbrewery Bangalore,5.0,User-3048280,Alexandrina Namrata
3,Brewklyn Microbrewery Bangalore,4.0,User-36088032,Abhishek Sharma
4,Brewklyn Microbrewery Bangalore,4.0,User-407847,Nitin Hajela (TheFoodieTales.com)


In [10]:
user_item_matrix = zomato.pivot_table(index='user_id',columns='rest_name',values='rating')
user_item_matrix

rest_name,1131 Bar + Kitchen Bangalore,99 Pancakes Bangalore,Arbor Brewing Company Bangalore,Arogya Ahaara Bangalore,Avista Cafe Pizzeria Bangalore,Barbecue by Punjab Grill Bangalore,BelgYum Bangalore,Big Wong XL Bangalore,Bloomsbury's Global Kitchen & Bakehouse Bangalore,Bologna Bangalore,...,The Globe Grub Bangalore,The Konkan - Seafood Restaurant Bangalore,The Old Fashioned Bar Bangalore,The Permit Room Bangalore,The Pizza Bakery Bangalore,The Terrace at Gilly's Redefined Bangalore,Toast & Tonic Bangalore,Toit Bangalore,Yauatcha Bangalore,eat.fit Bangalore
user_id,,,,,,,,,,,,,,,,,,,,,
User-10009821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-1001535,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-1001893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-1002382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-1002399,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-1002419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
User-10029081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User-1003034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
User-1003102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
ui_matrix = user_item_matrix.apply(lambda v:v-v.mean(),axis=1).fillna(0)

In [12]:
ui_matrix

rest_name,1131 Bar + Kitchen Bangalore,99 Pancakes Bangalore,Arbor Brewing Company Bangalore,Arogya Ahaara Bangalore,Avista Cafe Pizzeria Bangalore,Barbecue by Punjab Grill Bangalore,BelgYum Bangalore,Big Wong XL Bangalore,Bloomsbury's Global Kitchen & Bakehouse Bangalore,Bologna Bangalore,...,The Globe Grub Bangalore,The Konkan - Seafood Restaurant Bangalore,The Old Fashioned Bar Bangalore,The Permit Room Bangalore,The Pizza Bakery Bangalore,The Terrace at Gilly's Redefined Bangalore,Toast & Tonic Bangalore,Toit Bangalore,Yauatcha Bangalore,eat.fit Bangalore
user_id,,,,,,,,,,,,,,,,,,,,,
User-10009821,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
User-1001535,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
User-1001893,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
User-1002382,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
User-1002399,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
User-1002419,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
User-10029081,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
User-1003034,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.75,0.0,0.0,0.000000,0.0,0.0,0.0
User-1003102,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0


In [13]:
sim_matrix = pd.DataFrame(cosine_similarity(ui_matrix),index=ui_matrix.index,columns=ui_matrix.index)

In [14]:
neighs = sim_matrix['User-1826583'].drop('User-1826583').sort_values(ascending=False).head(7).index
neighs

Index(['User-27737095', 'User-35039943', 'User-15671029', 'User-11718651',
       'User-428014', 'User-45563960', 'User-36088032'],
      dtype='object', name='user_id')

In [15]:
preds = pd.DataFrame(columns=['restaurant','predicted_rating'])
for rest_name,rating in user_item_matrix.loc['User-1826583'].items():
    if pd.isnull(rating):
        rest_ratings =user_item_matrix.loc[neighs,rest_name]
        avg_rest_rating =rest_ratings.mean()
        preds =preds.append({'restaurant':rest_name,'predicted_rating':avg_rest_rating},ignore_index=True)
preds.sort_values('predicted_rating',ascending=False).head(3)

,restaurant,predicted_rating
8,Bloomsbury's Global Kitchen & Bakehouse Bangalore,5.0
16,Chikkanna Tiffin Room Bangalore,5.0
39,Milano Ice Cream Bangalore,5.0


# Creating a function

In [16]:
def ubcf(userid):
    neighs_userid=sim_matrix[userid].drop(userid).sort_values(ascending=False).head(7).index
    preds =pd.DataFrame(columns=['rest_name','predicted_rating'])
    for rest_name,rating in user_item_matrix.loc[userid].items():
        if pd.isnull(rating):
            rest_ratings =user_item_matrix.loc[neighs_userid,rest_name]
            avg_rest_rating =rest_ratings.mean()
            preds =preds.append({'rest_name':rest_name,'predicted_rating':avg_rest_rating},ignore_index=True)

    movies_recom =preds.sort_values('predicted_rating',ascending=False).head(3)['rest_name']
    return movies_recom

#  Sample recommendations for new user

In [18]:
ubcf('User-31021666')

9              Bologna Bangalore
39              Maziga Bangalore
40    Milano Ice Cream Bangalore
Name: rest_name, dtype: object

In [21]:
ubcf('User-407847')

0       1131 Bar + Kitchen Bangalore
4     Avista Cafe Pizzeria Bangalore
69          TBC Sky Lounge Bangalore
Name: rest_name, dtype: object

In [22]:
ubcf('User-36088032')

13                 CTR Bangalore
81    The Pizza Bakery Bangalore
52    Output Bengaluru Bangalore
Name: rest_name, dtype: object